In [50]:
%matplotlib inline
import os
import sys
import glob
import math
import codecs
import warnings
import numpy as np
from scipy import stats
import nilearn.datasets
import nilearn.connectome
from sklearn.svm import SVC
from itertools import count
from functools import partial
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.manifold import TSNE
from scipy.stats import normaltest
from collections import defaultdict
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation as animation
from sklearn.cross_validation import KFold
from sklearn.covariance import GraphLassoCV
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from statsmodels.stats.multitest import multipletests
from sklearn.cross_validation import StratifiedKFold, cross_val_score
from statsmodels.stats.multicomp import (pairwise_tukeyhsd,MultiComparison)

warnings.filterwarnings('ignore')

In [51]:
def apply_svd(X, dim):
    svd = TruncatedSVD(n_components=dim)
    svd.fit(X)
    Xdim = svd.transform(X)
    return Xdim

In [52]:
def parse_names(filenames):
  names = []
  for filename in filenames:
    name = filename.replace('/Users/admin/Documents/Stanford/Research/Neuro-Anesthesia/data/Stanford-Anesthesia-Preprocessed/','')
    name = name.replace('/session_1/rest_1/rest.nii.gz','')
    name = name.replace('Exposed/','E_')
    name = name.replace('Control/', 'C_')
    names.append(name)

  return names

In [53]:
def dump_data(kind, ids, data):
  label_file = codecs.open(kind+'_labels.txt', "w", "utf-8")
  file = codecs.open(kind+'_embeddings.txt', "w", "utf-8")
  for id, vect in zip(ids, data):
    vect = map(str, vect.tolist())
    vect_str = id+' '+(' '.join(vect))+'\n'
    label_file.write(id+'\n')
    file.write(vect_str)
  file.close()
  label_file.close()
  return

In [54]:
def read_dataset():
    atlas = nilearn.datasets.fetch_atlas_msdl()

    test_subject_path = '/Users/admin/Documents/Stanford/Research/Neuro-Anesthesia/data/Stanford-Anesthesia/Exposed/'
    control_subject_path = '/Users/admin/Documents/Stanford/Research/Neuro-Anesthesia/data/Stanford-Anesthesia/Control/'

    test_subjects_resting_files = glob.glob(test_subject_path+ '*/session_1/rest_1/*.gz')
    test_subject_labels = np.ones(len(test_subjects_resting_files))

    control_subjects_resting_files = glob.glob(control_subject_path+ '*/session_1/rest_1/*.gz')
    control_subject_labels = np.zeros(len(control_subjects_resting_files))

    test_ids = parse_names(test_subjects_resting_files)
    control_ids = parse_names(control_subjects_resting_files)

    test_subjects_names = ['E'+str(directories+1) for directories in range(12)]
    control_subjects_names = ['C'+str(directories+1) for directories in range(12)]

    resting_files = test_subjects_resting_files + control_subjects_resting_files
    labels = np.concatenate((test_subject_labels, control_subject_labels), axis=0)
    names = np.concatenate((test_subjects_names, control_subjects_names), axis=0)
    ids = np.concatenate((test_ids, control_ids), axis=0)

    return atlas, resting_files, labels, names, ids

In [55]:
def extract_time_series(resting_files, labels, atlas):
    ######################################################################
    # Extract regions time series signals
    import nilearn.input_data
    masker = nilearn.input_data.NiftiMapsMasker(
        atlas.maps, resampling_target="maps", detrend=True,
        low_pass=None, high_pass=None, t_r=2.5, standardize=False,
        memory='nilearn_cache', memory_level=1)
    subjects = []
    for func_file, phenotypic in zip(resting_files, labels):
        time_series = masker.fit_transform(func_file)
        subjects.append(time_series)
    
    return subjects

In [56]:
def calculate_functional_connectivity_matrices(subjects, kind):
    
    conn_measure = nilearn.connectome.ConnectivityMeasure(kind=kind)
    individual_connectivity_matrices = conn_measure.fit_transform(subjects)
    
    return individual_connectivity_matrices

In [57]:
def get_atlas_labels(atlas):
    atlas_labels = []

    for label in atlas.labels:
      atlas_labels.append(label.decode('UTF-8'))
    return atlas_labels

In [86]:
def split_connectivity_matrices(individual_connectivity_matrices, labels, num_fold):
    test_subjects_indices = [i for i in range(len(labels)) if labels[i] == 1]
    control_subjects_indices = [i for i in range(len(labels)) if labels[i] == 0]

    np.random.seed(29051453)
    np.random.shuffle(test_subjects_indices)
    np.random.shuffle(control_subjects_indices)
        
    test_subjects_matrices = individual_connectivity_matrices[test_subjects_indices,:,:]
    control_subjects_matrices = individual_connectivity_matrices[control_subjects_indices,:,:]
    
    fold_size = int(len(test_subjects_indices)/num_fold)
    print('fold_size',fold_size)
    splitted_datasets=[]
    for test_start_index in range(0, len(test_subjects_indices), fold_size):
        selected_test_indices = np.r_[test_start_index:test_start_index+fold_size]
        selected_train_indices = [x for x in np.r_[0:len(test_subjects_indices)] if x not in selected_test_indices] 
        
        test_subjects_matrices_train = test_subjects_matrices[selected_train_indices, :, :]
        control_subjects_matrices_train = control_subjects_matrices[selected_train_indices, :, :]

        test_subjects_matrices_test = test_subjects_matrices[selected_test_indices, :, :]
        control_subjects_matrices_test = control_subjects_matrices[selected_test_indices, :, :]

        individual_connectivity_matrices_train = np.concatenate((control_subjects_matrices_train, test_subjects_matrices_train), axis=0)
        individual_connectivity_matrices_test = np.concatenate((control_subjects_matrices_test, test_subjects_matrices_test), axis=0)

        train_size = len(selected_train_indices)
        test_size = len(selected_test_indices)

        labels_test = np.concatenate((np.zeros(test_size), np.ones(test_size)), axis=0)
        labels_train = np.concatenate((np.zeros(train_size), np.ones(train_size)), axis=0)

        splitted_dataset = {

            'individual_connectivity_matrices':individual_connectivity_matrices,
            'individual_connectivity_matrices_train': individual_connectivity_matrices_train,
            'individual_connectivity_matrices_test': individual_connectivity_matrices_test,
            'labels_train':labels_train,
            'labels_test':labels_test,
            'control_subjects_matrices':control_subjects_matrices,

            'test_subjects_matrices':test_subjects_matrices,
            'control_subjects_matrices_train':control_subjects_matrices_train,
            'control_subjects_matrices_test':control_subjects_matrices_test,
            'test_subjects_matrices_train': test_subjects_matrices_train,
            'test_subjects_matrices_test': test_subjects_matrices_test,
            
            'selected_train_indices':selected_train_indices,
            'selected_test_indices':selected_test_indices

        }
        splitted_datasets.append(splitted_dataset)
    return splitted_datasets

In [87]:
def select_features_with_ttest(splitted_dataset, atlas):
    t_mat, prob_mat = stats.ttest_ind(splitted_dataset['control_subjects_matrices_train'], splitted_dataset['test_subjects_matrices_train'], equal_var=False)
    
    t_test_rejection_threshold = 0.05
    
    reject_indexes_mat = np.argwhere(prob_mat < t_test_rejection_threshold)
    
    if reject_indexes_mat.shape[0] == 0:
        print('no significant change in any region pairs')
        sys.exit(-1)
    
    reject_indexes_mat = np.squeeze(reject_indexes_mat)
    
    selected_feature_set_train = np.zeros(shape=splitted_dataset['individual_connectivity_matrices_train'].shape, dtype=splitted_dataset['individual_connectivity_matrices_train'].dtype)
    train_features = []
    for x, y in reject_indexes_mat:
        
        if x >= y: # ignore lower triangular part of the matrix
            continue
        selected_feature_set_train[:, x,y] = splitted_dataset['individual_connectivity_matrices_train'][:, x, y]
        train_features.append(splitted_dataset['individual_connectivity_matrices_train'][:, x, y])
    
    train_features = np.vstack(train_features)
    train_features = np.transpose(train_features)
    
    selected_feature_set_test = np.zeros(shape=splitted_dataset['individual_connectivity_matrices_test'].shape, dtype=splitted_dataset['individual_connectivity_matrices_test'].dtype)
    test_features = []
    for x, y in reject_indexes_mat:
        
        if x >= y: # ignore lower triangular part of the matrix
            continue
        selected_feature_set_test[:, x,y] = splitted_dataset['individual_connectivity_matrices_test'][:, x, y]
        test_features.append(splitted_dataset['individual_connectivity_matrices_test'][:, x, y])
    
    test_features = np.vstack(test_features)
    test_features = np.transpose(test_features)
    
    selected_feature_set_test = selected_feature_set_test.reshape(selected_feature_set_test.shape[0], -1)
    selected_feature_set_train = selected_feature_set_train.reshape(selected_feature_set_train.shape[0], -1)
    
    
    prob_mat_plot = np.zeros(shape=prob_mat.shape, dtype=np.int32)
    
    prob_mat_plot[:] = 0
        
    for x, y in reject_indexes_mat:
        prob_mat_plot[x, y] = 1
    
    reject_indexes_mat = prob_mat_plot
    
    selected_feature_set ={
        'train': train_features,
        'test':test_features
    }

    stats_results = {
        'reject_indexes_mat':reject_indexes_mat
    }
    return selected_feature_set, stats_results

In [88]:
def select_features_with_normality_test(splitted_dataset, atlas):
    
    rejection_threshold = 0.05
    
    total_count = 0.0
    normal_count = 0.0
    no_normal_count = 0.0
    train_features = []
    test_features = []
    reject_indexes_mat = np.zeros(shape=splitted_dataset['individual_connectivity_matrices_train'].shape[1:3], dtype=np.int32)
    print('reject_indexes_mat.shape',reject_indexes_mat.shape)
    for x in range(splitted_dataset['individual_connectivity_matrices_train'].shape[1]):
        for y in range(splitted_dataset['individual_connectivity_matrices_train'].shape[1]):
        
            if x >= y: # ignore lower triangular part of the matrix
                continue
            
            total_count += 1
            
            if splitted_dataset['individual_connectivity_matrices_train'].shape[0] == 12:
                #Raised an exception 'skewtest is not valid with less than 8 samples' when sample size = 6
                #Hence we assume features are non-Gaussian in this case
                normality_prob_control = 1.0
                normality_prob_test = 1.0
                
            else:
                normality_t_control, normality_prob_control = normaltest(splitted_dataset['control_subjects_matrices_train'][:, x, y])
                normality_t_test, normality_prob_test = normaltest(splitted_dataset['test_subjects_matrices_train'][:, x, y])

            if normality_prob_control <=rejection_threshold and normality_prob_test <= rejection_threshold:
                #print('significant x,y', x,y, normality_prob_control, normality_prob_test)
                train_features.append(splitted_dataset['individual_connectivity_matrices_train'][:, x, y])
                test_features.append(splitted_dataset['individual_connectivity_matrices_test'][:, x, y])
                reject_indexes_mat[x,y] = 1
                normal_count += 1
            else:
                ks_value, ks_prob = stats.ks_2samp(splitted_dataset['control_subjects_matrices_train'][:, x, y], splitted_dataset['test_subjects_matrices_train'][:, x, y])
                if ks_prob <= rejection_threshold:
                    #print('significant x,y', x,y, ks_prob)
                    no_normal_count += 1
                    train_features.append(splitted_dataset['individual_connectivity_matrices_train'][:, x, y])
                    test_features.append(splitted_dataset['individual_connectivity_matrices_test'][:, x, y])
                    reject_indexes_mat[x,y] = 1
                
    print('Normal ratio:', normal_count/total_count)
    print('No Normal ratio:', no_normal_count/total_count)
    
    
    train_features = np.vstack(train_features)
    train_features = np.transpose(train_features)
    
    print('train_features', train_features.shape)
    
   
    test_features = np.vstack(test_features)
    test_features = np.transpose(test_features)
    
    print('test_features', test_features.shape)
    
    selected_feature_set ={
        'train': train_features,
        'test':test_features
    }

    stats_results = {
        'reject_indexes_mat':reject_indexes_mat
    }
    return selected_feature_set, stats_results

In [89]:
def select_features_all(splitted_dataset, atlas):
    #select all features
    
    selected_feature_set_train = np.zeros(shape=splitted_dataset['individual_connectivity_matrices_train'].shape, dtype=splitted_dataset['individual_connectivity_matrices_train'].dtype)
    train_features = []
    for x in range(splitted_dataset['individual_connectivity_matrices_train'].shape[1]):
        for y in range(splitted_dataset['individual_connectivity_matrices_train'].shape[1]):
            if x >= y: # ignore lower triangular part of the matrix
                continue
            selected_feature_set_train[:, x,y] = splitted_dataset['individual_connectivity_matrices_train'][:, x, y]
            train_features.append(splitted_dataset['individual_connectivity_matrices_train'][:, x, y])
    
    train_features = np.vstack(train_features)
    train_features = np.transpose(train_features)
    
    selected_feature_set_test = np.zeros(shape=splitted_dataset['individual_connectivity_matrices_test'].shape, dtype=splitted_dataset['individual_connectivity_matrices_test'].dtype)
    test_features = []
    for x in range(splitted_dataset['individual_connectivity_matrices_train'].shape[1]):
        for y in range(splitted_dataset['individual_connectivity_matrices_train'].shape[1]):
        
            if x >= y: # ignore lower triangular part of the matrix
                continue
            selected_feature_set_test[:, x,y] = splitted_dataset['individual_connectivity_matrices_test'][:, x, y]
            test_features.append(splitted_dataset['individual_connectivity_matrices_test'][:, x, y])
    
    test_features = np.vstack(test_features)
    test_features = np.transpose(test_features)
    
    selected_feature_set_test = selected_feature_set_test.reshape(selected_feature_set_test.shape[0], -1)
    selected_feature_set_train = selected_feature_set_train.reshape(selected_feature_set_train.shape[0], -1)
    
    reject_indexes_mat = np.ones(shape=splitted_dataset['individual_connectivity_matrices_train'].shape[1:3], dtype=np.int32)
    
    print('train_features.shape', train_features.shape)
    print('test_features.shape', test_features.shape)
    selected_feature_set ={
        'train': train_features,
        'test':test_features
    }

    stats_results = {
        'reject_indexes_mat':reject_indexes_mat
    }
    return selected_feature_set, stats_results

In [90]:
def plot_decision_boundary(X, y, names, kind):

  label_to_number = defaultdict(partial(next, count(1)))
  ytmp = np.array(y)
  index_test = ytmp == 1
  index_control = ytmp == 0
  
  fig, ax = plt.subplots()

  ax_control = ax.scatter(X[index_control, 0], X[index_control, 1], c='blue', s=50*1*1, cmap=plt.cm.Spectral)
  ax_exposed = ax.scatter(X[index_test, 0], X[index_test, 1], c='red', s=50*2*2, cmap=plt.cm.Spectral)

  plt.legend((ax_control, ax_exposed),('Control', 'Exposed'), loc=1, scatterpoints = 1)
  plt.suptitle(kind + " plot")
  plt.savefig(kind+'_anesthesia_visualization.png')   # save the figure to file

In [91]:
def print_and_plot_selected_features(selected_feature_set, stats_results, labels, atlas, names, kind):
    
    reject_indexes_mat_labels = [(atlas.labels[ind[0]], atlas.labels[ind[1]]) for ind in stats_results['reject_indexes_mat'] if ind[0] < ind[1]]
    
    
    prob_mat_plot = stats_results['reject_indexes_mat']
        
    
    atlas_labels = get_atlas_labels (atlas)
    
    print ('------- Selected region pairs in matrix format --------')
    
    plt.figure(figsize=(10, 10))
    # Mask out the major diagonal
    np.fill_diagonal(prob_mat_plot, 0)
    plt.imshow(prob_mat_plot, interpolation="nearest", cmap="Reds",vmax=1, vmin=0)
    plt.colorbar()
    # And display the labels
    x_ticks = plt.xticks(range(len(atlas.labels)), atlas_labels, rotation=90)
    y_ticks = plt.yticks(range(len(atlas.labels)), atlas_labels)
    plt.show()
    
    x1 = apply_svd(selected_feature_set, 2)
    x2 = TSNE(n_components=2, verbose=0).fit_transform(selected_feature_set)
    plot_decision_boundary(x1, labels, names, kind)

In [92]:
def evaluate_accuracy(actual_set, predicted_set):
    correct_count  = 0.0
    total_count = 0.0
    for (actual, predicted) in zip(actual_set, predicted_set):
        total_count += 1
        if actual == predicted:
            correct_count += 1
    accuracy = correct_count/total_count
    return accuracy

In [93]:
def classify_grid_search_cv(selected_feature_set, splitted_dataset):
    
    X_train = selected_feature_set['train']
    y_train = splitted_dataset['labels_train']
    
    X_test = selected_feature_set['test']
    y_test = splitted_dataset['labels_test']
    
    # Set the parameters by cross-validation
    tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],'C': [0.001, 0.01, 0.1, 10, 100, 1000]},
                        {'kernel': ['linear'], 'C': [0.001, 0.01, 0.1, 10, 100, 1000]}]

    #Experiment different success metrics
    #scores = ['precision_macro', 'recall_macro', 'f1_macro', 'accuracy']
    scores = ['accuracy']
    for score in scores:
        print("# Tuning hyper-parameters for %s" % score)
        
        clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=4, scoring=score)
        clf.fit(X_train, y_train)
        print("Best parameters set found on development set:")
        print()
        print(clf.best_params_)
        
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean, std * 2, params))
        
        y_predicted = clf.predict(X_test)
        
        print(classification_report(y_test, y_predicted))
        

In [94]:
def classify(selected_feature_set, splitted_dataset, names):
    
    X_train = selected_feature_set['train']
    y_train = splitted_dataset['labels_train']
    
    X_test = selected_feature_set['test']
    y_test = splitted_dataset['labels_test']
    
    #Experiment different SVM kernels
    C=0.001
    gamma= 0.001
    svc = SVC(kernel='rbf',C=C, gamma=gamma)
    #svc = LinearSVC(C=C)
    
    svc.fit(X_train, y_train)
    
    result_train = svc.predict(selected_feature_set['train'])
    print('----------------------------')
    print('actual train labels', splitted_dataset['labels_train'])
    print('predicted train labels', result_train)
    train_accuracy = evaluate_accuracy(splitted_dataset['labels_train'], result_train)
    print('train_accuracy', train_accuracy)
    
    print('----------------------------')
    y_predicted = svc.predict(X_test)
    print('actual test labels', y_test)
    print('predicted test labels', y_predicted)
    
    test_accuracy = evaluate_accuracy(y_test, y_predicted)
    print('test_accuracy', test_accuracy)
    return train_accuracy, test_accuracy
    


In [133]:
def analyze_functional_connectivity(kind, num_fold, feature_selection):
    
    print ('************* ' + kind + ' analysis *************')
    atlas, resting_files, labels, names, ids = read_dataset()
    
    subjects = extract_time_series(resting_files, labels, atlas)
    
    individual_connectivity_matrices = calculate_functional_connectivity_matrices (subjects, kind)
    
    #Experiment different values of num_fold such that num_fold in {2,3,4,6,12}
    splitted_datasets = split_connectivity_matrices(individual_connectivity_matrices, labels, num_fold=num_fold)
    
    train_accuracy_values = []
    test_accuracy_values = []
    for splitted_dataset in splitted_datasets:
        print('********************** New Split ***********************')
        print('selected_train_indices', splitted_dataset['selected_train_indices'])
        print('selected_test_indices', splitted_dataset['selected_test_indices'])
        
        #Experiment feature selection with normality test, pure t-test or selecting all features
        #selected_feature_set, stats_results = select_features_with_ttest(splitted_dataset, atlas)
        if feature_selection == True:
            selected_feature_set, stats_results = select_features_with_normality_test(splitted_dataset, atlas)
        else:
            selected_feature_set, stats_results = select_features_all(splitted_dataset, atlas)
        
        #Experiment visualization of the training and test datasets
        #print_and_plot_selected_features(selected_feature_set['train'], stats_results, splitted_dataset['labels_train'], atlas, names, kind+' train')
        #print_and_plot_selected_features(selected_feature_set['test'], stats_results, splitted_dataset['labels_test'], atlas, names, kind+' test')

        #Experiment accuracy performance of different hyperparameter values
        #classify_grid_search_cv(selected_feature_set, splitted_dataset)

        train_accuracy, test_accuracy = classify(selected_feature_set, splitted_dataset, names)
        train_accuracy_values.append(train_accuracy)
        test_accuracy_values.append(test_accuracy)
    
    train_accuracy_values = np.asarray(train_accuracy_values)
    test_accuracy_values = np.asarray(test_accuracy_values)
    
    print('--------------------')
    print('CV train_accuracy_values', train_accuracy_values)
    print('CV mean train accuracy: %1.2f, std dev %1.2f:' %(train_accuracy_values.mean(), train_accuracy_values.std()))
    print('--------------------')
    print('CV test_accuracy_values', test_accuracy_values)
    print('CV mean test accuracy: %1.2f, std dev %1.2f:' %(test_accuracy_values.mean(), test_accuracy_values.std()))


Results of k-fold experiments with feature selection disabled

In [117]:
analyze_functional_connectivity(kind='partial correlation', num_fold=2, feature_selection=False)

************* partial correlation analysis *************
fold_size 6
********************** New Split ***********************
selected_train_indices [6, 7, 8, 9, 10, 11]
selected_test_indices [0 1 2 3 4 5]
train_features.shape (12, 741)
test_features.shape (12, 741)
----------------------------
actual train labels [ 0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.]
predicted train labels [ 0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.]
train_accuracy 1.0
----------------------------
actual test labels [ 0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.]
predicted test labels [ 0.  1.  1.  0.  1.  0.  1.  0.  1.  1.  1.  0.]
test_accuracy 0.5833333333333334
********************** New Split ***********************
selected_train_indices [0, 1, 2, 3, 4, 5]
selected_test_indices [ 6  7  8  9 10 11]
train_features.shape (12, 741)
test_features.shape (12, 741)
----------------------------
actual train labels [ 0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.]
predicted train labels [ 1.  0.  0.  0

In [116]:
analyze_functional_connectivity(kind='partial correlation', num_fold=3, feature_selection=False)

************* partial correlation analysis *************
fold_size 4
********************** New Split ***********************
selected_train_indices [4, 5, 6, 7, 8, 9, 10, 11]
selected_test_indices [0 1 2 3]
train_features.shape (16, 741)
test_features.shape (8, 741)
----------------------------
actual train labels [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.]
predicted train labels [ 0.  0.  1.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.]
train_accuracy 0.9375
----------------------------
actual test labels [ 0.  0.  0.  0.  1.  1.  1.  1.]
predicted test labels [ 1.  0.  0.  0.  1.  0.  0.  1.]
test_accuracy 0.625
********************** New Split ***********************
selected_train_indices [0, 1, 2, 3, 8, 9, 10, 11]
selected_test_indices [4 5 6 7]
train_features.shape (16, 741)
test_features.shape (8, 741)
----------------------------
actual train labels [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.]
predicted train labels [ 1.  0.  0

In [115]:
analyze_functional_connectivity(kind='partial correlation', num_fold=4, feature_selection=False)

************* partial correlation analysis *************
fold_size 3
********************** New Split ***********************
selected_train_indices [3, 4, 5, 6, 7, 8, 9, 10, 11]
selected_test_indices [0 1 2]
train_features.shape (18, 741)
test_features.shape (6, 741)
----------------------------
actual train labels [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
predicted train labels [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
train_accuracy 0.9444444444444444
----------------------------
actual test labels [ 0.  0.  0.  1.  1.  1.]
predicted test labels [ 1.  0.  0.  1.  0.  0.]
test_accuracy 0.5
********************** New Split ***********************
selected_train_indices [0, 1, 2, 6, 7, 8, 9, 10, 11]
selected_test_indices [3 4 5]
train_features.shape (18, 741)
test_features.shape (6, 741)
----------------------------
actual train labels [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
predicted trai

In [118]:
analyze_functional_connectivity(kind='partial correlation', num_fold=6, feature_selection=False)

************* partial correlation analysis *************
fold_size 2
********************** New Split ***********************
selected_train_indices [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
selected_test_indices [0 1]
train_features.shape (20, 741)
test_features.shape (4, 741)
----------------------------
actual train labels [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.]
predicted train labels [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.]
train_accuracy 0.95
----------------------------
actual test labels [ 0.  0.  1.  1.]
predicted test labels [ 1.  1.  1.  0.]
test_accuracy 0.25
********************** New Split ***********************
selected_train_indices [0, 1, 4, 5, 6, 7, 8, 9, 10, 11]
selected_test_indices [2 3]
train_features.shape (20, 741)
test_features.shape (4, 741)
----------------------------
actual train labels [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.]
predicted trai

In [132]:
analyze_functional_connectivity(kind='partial correlation', num_fold=12, feature_selection=False)

************* partial correlation analysis *************
fold_size 1
********************** New Split ***********************
selected_train_indices [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
selected_test_indices [0]
train_features.shape (22, 741)
test_features.shape (2, 741)
# Tuning hyper-parameters for accuracy
Best parameters set found on development set:

{'kernel': 'rbf', 'gamma': 0.001, 'C': 0.001}
0.682 (+/-0.254) for {'kernel': 'rbf', 'gamma': 0.001, 'C': 0.001}
0.682 (+/-0.254) for {'kernel': 'rbf', 'gamma': 0.0001, 'C': 0.001}
0.682 (+/-0.254) for {'kernel': 'rbf', 'gamma': 0.001, 'C': 0.01}
0.682 (+/-0.254) for {'kernel': 'rbf', 'gamma': 0.0001, 'C': 0.01}
0.682 (+/-0.254) for {'kernel': 'rbf', 'gamma': 0.001, 'C': 0.1}
0.682 (+/-0.254) for {'kernel': 'rbf', 'gamma': 0.0001, 'C': 0.1}
0.682 (+/-0.254) for {'kernel': 'rbf', 'gamma': 0.001, 'C': 10}
0.682 (+/-0.254) for {'kernel': 'rbf', 'gamma': 0.0001, 'C': 10}
0.682 (+/-0.254) for {'kernel': 'rbf', 'gamma': 0.001, 'C': 100}
0.68

Results of k-fold experiments with feature selection enabled

In [125]:
analyze_functional_connectivity(kind='partial correlation', num_fold=2, feature_selection=True)

************* partial correlation analysis *************
fold_size 6
********************** New Split ***********************
selected_train_indices [6, 7, 8, 9, 10, 11]
selected_test_indices [0 1 2 3 4 5]
reject_indexes_mat.shape (39, 39)
Normal ratio: 0.0
No Normal ratio: 0.04183535762483131
train_features (12, 31)
test_features (12, 31)
----------------------------
actual train labels [ 0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.]
predicted train labels [ 0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.]
train_accuracy 1.0
----------------------------
actual test labels [ 0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.]
predicted test labels [ 1.  1.  1.  1.  1.  1.  1.  0.  1.  1.  1.  0.]
test_accuracy 0.3333333333333333
********************** New Split ***********************
selected_train_indices [0, 1, 2, 3, 4, 5]
selected_test_indices [ 6  7  8  9 10 11]
reject_indexes_mat.shape (39, 39)
Normal ratio: 0.0
No Normal ratio: 0.016194331983805668
train_features (12, 12)
test_feat

In [127]:
analyze_functional_connectivity(kind='partial correlation', num_fold=3, feature_selection=True)

************* partial correlation analysis *************
fold_size 4
********************** New Split ***********************
selected_train_indices [4, 5, 6, 7, 8, 9, 10, 11]
selected_test_indices [0 1 2 3]
reject_indexes_mat.shape (39, 39)
Normal ratio: 0.059379217273954114
No Normal ratio: 0.10256410256410256
train_features (16, 120)
test_features (8, 120)
----------------------------
actual train labels [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.]
predicted train labels [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  0.  1.]
train_accuracy 0.9375
----------------------------
actual test labels [ 0.  0.  0.  0.  1.  1.  1.  1.]
predicted test labels [ 1.  0.  0.  0.  1.  0.  0.  1.]
test_accuracy 0.625
********************** New Split ***********************
selected_train_indices [0, 1, 2, 3, 8, 9, 10, 11]
selected_test_indices [4 5 6 7]
reject_indexes_mat.shape (39, 39)
Normal ratio: 0.07827260458839407
No Normal ratio: 0.08097165991902834
train_feat

In [128]:
analyze_functional_connectivity(kind='partial correlation', num_fold=4, feature_selection=True)

************* partial correlation analysis *************
fold_size 3
********************** New Split ***********************
selected_train_indices [3, 4, 5, 6, 7, 8, 9, 10, 11]
selected_test_indices [0 1 2]
reject_indexes_mat.shape (39, 39)
Normal ratio: 0.08097165991902834
No Normal ratio: 0.03508771929824561
train_features (18, 86)
test_features (6, 86)
----------------------------
actual train labels [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
predicted train labels [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  0.  1.  1.  0.]
train_accuracy 0.8888888888888888
----------------------------
actual test labels [ 0.  0.  0.  1.  1.  1.]
predicted test labels [ 1.  0.  0.  0.  0.  0.]
test_accuracy 0.3333333333333333
********************** New Split ***********************
selected_train_indices [0, 1, 2, 6, 7, 8, 9, 10, 11]
selected_test_indices [3 4 5]
reject_indexes_mat.shape (39, 39)
Normal ratio: 0.10256410256410256
No Normal ratio: 0.0256

In [129]:
analyze_functional_connectivity(kind='partial correlation', num_fold=6, feature_selection=True)

************* partial correlation analysis *************
fold_size 2
********************** New Split ***********************
selected_train_indices [2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
selected_test_indices [0 1]
reject_indexes_mat.shape (39, 39)
Normal ratio: 0.11470985155195682
No Normal ratio: 0.05263157894736842
train_features (20, 124)
test_features (4, 124)
----------------------------
actual train labels [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.]
predicted train labels [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  0.  1.
  1.  1.]
train_accuracy 0.9
----------------------------
actual test labels [ 0.  0.  1.  1.]
predicted test labels [ 1.  1.  0.  0.]
test_accuracy 0.0
********************** New Split ***********************
selected_train_indices [0, 1, 4, 5, 6, 7, 8, 9, 10, 11]
selected_test_indices [2 3]
reject_indexes_mat.shape (39, 39)
Normal ratio: 0.08232118758434548
No Normal ratio: 0.048582995951417005
train_fea

In [130]:
analyze_functional_connectivity(kind='partial correlation', num_fold=12, feature_selection=True)

************* partial correlation analysis *************
fold_size 1
********************** New Split ***********************
selected_train_indices [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
selected_test_indices [0]
reject_indexes_mat.shape (39, 39)
Normal ratio: 0.09986504723346828
No Normal ratio: 0.07017543859649122
train_features (22, 126)
test_features (2, 126)
----------------------------
actual train labels [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.]
predicted train labels [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  1.  1.  1.  1.
  1.  0.  0.  1.]
train_accuracy 0.8636363636363636
----------------------------
actual test labels [ 0.  1.]
predicted test labels [ 1.  1.]
test_accuracy 0.5
********************** New Split ***********************
selected_train_indices [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
selected_test_indices [1]
reject_indexes_mat.shape (39, 39)
Normal ratio: 0.12280701754385964
No Normal ratio: 0.06612685560